In [1]:
library(MASS)
library(fitHeavyTail)
library(xts)
library(quantmod)
library(igraph)
library(readr)
library(spectralGraphTopology)
library(combinat)
library(ggplot2)
library(reshape2)
library(ktvgraph)

library(Matrix)
library(LaplacesDemon)
library(expm)

source("R/utils.R")
source("R/learn_kcomp_heavytail_TV_graph_online.R")

options(warn = -1)

Warning message:
"package 'MASS' was built under R version 4.5.1"
Warning message:
"package 'fitHeavyTail' was built under R version 4.5.1"
Loading required package: zoo


Attaching package: 'zoo'


The following objects are masked from 'package:base':

    as.Date, as.Date.numeric


Warning message:
"package 'quantmod' was built under R version 4.5.1"
Loading required package: TTR

Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 

Warning message:
"package 'igraph' was built under R version 4.5.1"

Attaching package: 'igraph'


The following objects are masked from 'package:stats':

    decompose, spectrum


The following object is masked from 'package:base':

    union



Attaching package: 'spectralGraphTopology'


The following object is masked from 'package:stats':

    D



Attaching package: 'combinat'


The following object is masked from 'package:utils':

    combn


Warning message:
"package 'ggplot2' was built under R version 

In [2]:
set.seed(42)


pdf(file = "Plots_TVGL_results.pdf")


# number of sectors
q <- 1
winLen <-  200

# number of stocks
r <- 10
#log_returns <- read.csv("log_rw_returns.csv")
log_returns <- read.csv("log_returns.csv")

Nday <- nrow(log_returns) 

Nwin <- Nday%/% winLen  


Nday <- nrow(log_returns) 

Nwin <- Nday%/% winLen  




#----------------------------
## Online TV graph learning (proposed)
forget_fac <- 0.1
data_frame <- scale(log_returns[1:winLen,1:r])
#data_frame <- rw_t[1:winLen,]

S_cov <- cor(data_frame)

w <- spectralGraphTopology:::w_init('naive', MASS::ginv(S_cov))

w0 <- w
A0 <- A(w)
A0 <- A0 / rowSums(A0)
w0 <- spectralGraphTopology:::Ainv(A0)
w0 = w0/sum(w0)

w_lagged <- w0

w_lagged <- 0

graphs_list <- vector("list", Nwin)

components_graph <- vector("list", Nwin)

accuracy_vec <- rep(0,Nwin)
accuracy_adj_vec <- rep(0,Nwin)
purity_vec <- rep(0,Nwin)
mod_g_vec <- rep(0,Nwin)
mod_gt_vec <- rep(0,Nwin)
balanced_vec <- rep(0,Nwin)
balanced_norm_vec <- rep(0,Nwin)
GINI_vec <- rep(0,Nwin)
rank_mat <- rep(0,Nwin)

#cat("init w: ",w,"\n")
#cat("init w_lagged: ", w_lagged,"\n")
#cat("init w0: ",w0,"\n")

Cluster = TRUE

# heavy_type = "gaussian"
# heavy_type = "student"

head(data_frame)

print(cat("w0", w0))

res_mvt <- fit_mvt(scale(data_frame), nu = "MLE-diag-resampled") #Estimate parameters of a multivariate Student's t
nu <- res_mvt$nu 


,A,AAL,ABBV,ABC,ABMD,ABT,ADM,AEE,AEP,AES
1,-0.2706328,-0.38601423,-0.28673217,1.1255779,-0.21450536,-0.009463173,0.5640626,1.008088039,0.7115951,0.6613222
2,0.2571223,0.71744928,-0.02113479,-1.0335841,0.06350252,-0.532614709,-1.4113041,0.516217296,0.2966966,-1.9450751
3,-2.9480683,-0.77586550,-0.21298577,-2.2636938,-1.86305946,-1.544144308,-1.3814046,-0.007945391,-1.1281159,-1.4168692
4,-0.7478337,-0.07485781,-1.72006894,-0.3407103,1.30967554,-1.346642996,-0.2592677,-0.783191987,-0.1941153,0.6960179
5,-1.1779783,0.72008389,-2.00563503,-2.8534563,0.12618782,0.099603967,-0.4219023,-0.049568420,0.7599951,-0.6576122
6,0.4004271,0.91310488,1.04670031,0.1698866,1.58846565,1.140201274,0.1275989,-0.365566785,-1.0048859,-1.7512894


w0 0.02479199 0.01695928 0.008705979 0.03772791 0.04709995 0.02619128 0.002494648 0 0.03320912 0 0.01864713 0.02598423 0.03416878 0.03912432 0.0007317251 0 0.02691992 0.02042875 0.03874434 0.03514684 0.01688847 0.04342322 0 0.003877046 0 0.08609423 0.002188217 0 0.01720016 0.0158432 0.01544395 0.004580683 0.009920442 0.01946893 0 0 0 0.02368696 0 0 0.01582319 0.07795123 0.1672792 0.008029461 0.0352252NULL


In [3]:

graphs_list <- learn_kcomp_heavytail_TV_graph_online(scale(data_frame),
                                                    k = q, # k the number of components of the graph. (Number of Clusters)
                                                    heavy_type = "student", # the statistical distribution of the data "gaussian" or "student".
                                                    nu = nu, # the degrees of freedom of the Student-t distribution. Must be a real number greater than 2.
                                                    sigma_e = exp(10), # hyperparameter that controls graph weight sparsity and time-consistency
                                                    w_lagged = w_lagged,
                                                    rho = 2, # ADMM hyperparameter.
                                                    d = 1, # the nodes' degrees. Either a vector or a single value.
                                                    w0 = w0, # initial value of the VAR coefficient
                                                    update_eta = TRUE, # whether to update eta during the optimization.
                                                    maxiter = 10,
                                                    verbose = TRUE)
graphs_list$lagrangian
cat("nu: ",nu,"\n")

T_n 200NULL
sigma_e 22026.47NULL
alpha 4.539993e-07NULL
beta 0.1NULL
u_func 1.045417NULL
u_func 10.83562NULL
u_func 14.0258NULL
u_func 13.91456NULL
u_func 12.91242NULL
u_func 11.5603NULL
u_func 10.35834NULL
u_func 10.26923NULL
u_func 10.36311NULL
u_func 10.42042NULL


[1] 40.13222 16.06274 12.44522 15.42977 15.92496 14.86558 14.17534 14.21498
 [9] 14.15039 13.93550

nu:  4.168752 


In [10]:
graphs_list$lagrangian

[1] 40.90830 17.71162 12.66376 15.92327 16.68371 15.72604 14.88240 14.87591
 [9] 14.80470 14.64092

In [15]:
graphs_list$weights

[1] 0.2259829 0.2828659 0.2057327 0.2854184 0.2342075 0.3031121 0.2366975
 [8] 0.2480988 0.2348278 0.2430563

In [ ]:
w <- c(0.1170013, 0.1527746, 0.04855342, 0.1085774, 0.1243674, 0.1150374, 0.09536096, 0.07961204, 0.08441811, 0.07429729)
L_n <- L(w)
print(L_n)
U <- eigen(L_n, symmetric = TRUE)$vectors
L <- eigen(L_n, symmetric = TRUE)$values
print("")
print(L)
print("")
print(U)


In [ ]:
# number of stocks
r <- 100 
stock_prices_orig <- readRDS("examples/stocks/sp500-data-2016-2020.rds")
stock_prices <- stock_prices_orig[1:1001,1:r]
log_returns <- diff(log(stock_prices), na.pad = FALSE)
write.csv(log_returns, file = "log_returns.csv", row.names = FALSE)